# Removing Erroneous Structures

This notebook shows how the 155 structures in tmQM that were removed when tmQM_wB97MV was generated were found and selected. Note that three additional structures, with CSD codes PIFFEP, NASBOA, and LIKBIS, were also removed due to nonconvergence of the DFT code. Of the 86,662 structures that converged, 155 were removed by the following procedure.

Data from the tmQM GitHub repository is used in this notebook, which can be installed at https://github.com/bbskjelstad/tmqm. One does not need to install that repository for this notebook to run, since the necessary data is included in this work. However, if one wanted to verify that the data included here is the same as in tmQM, they should first unzip the `.tar.gz` archives containing the structures in the tmqm folder, concatenate the two files, convert the format to an `extxyz`, and then remove the whitespace between adjacent entries in the file. This will result in the same file included at `all_data/tmQM/tmQM_combined_notcentered.extxyz`.

In [36]:
#import necessary packages
from ase.io import read
import numpy as np
from tqdm import tqdm, trange

In [2]:
#read extxyz containing all of the structures in tmQM
tmqm = read('../all_data/tmQM/tmQM_combined_notcentered.extxyz', index=slice(None), format='extxyz')

Above, we have read the file containing all of the structures in tmQM. In order to determine if there are erroneous structures, we first check if there are any carbons that have either zero or one neighboring atom, when this would violate valence rules.

We first define a function to determine, for each complex, for each carbon, the distances between that carbon and its neighbors, in order to find the carbon with the fewest neighbors, and thus bonds. Heuristics of 2 angstrom maximum bond distances for nonmetals with atomic numbers less than 30, and 2.5 angstroms for large nonmetals and metals were used as the cutoff for determining neighbors. Then, we run this function to find all of the structures that have carbons with zero or one neighbor, since they are likely to have erroneous structures.

In [3]:
nm_list = [1,  5,  6,  7,  8,  9, 14, 15, 16, 17, 33, 34, 35, 53] #all nonmetals in tmQM

def fewestBonds(atoms):
    nm_cutoff = 2 #if distance between atoms >2A, not considered neighbors for C (small nonmetals)
    m_cutoff = 2.5 #cutoff for metal bonds is 2.5A (and for larger nonmetals, atomic number >30)
    positions = atoms.get_positions() #get the positions of the atoms
    elements = atoms.get_atomic_numbers() #get the atomic numbers of the atoms
    num_neighbors = np.zeros(len(elements)) #initialize an array to store the number of neighbors for each atom
    for idx, elem1 in enumerate(elements): #for every atom
        if elem1 != 6: #only check the neighbors of the carbons, so skip if not a carbon
            continue
        pos1 = positions[idx] #get the positions of the carbon of interest
        for jdx, elem2 in enumerate(elements): #find the distance between the carbon and all other atoms
            pos2 = positions[jdx] #get the position of the other atom
            xdist = pos1[0] - pos2[0]
            ydist = pos1[1] - pos2[1]
            zdist = pos1[2] - pos2[2]
            dist = np.sqrt(xdist**2 + ydist**2 + zdist**2)
            if elem2 in nm_list: #determine if the other atom is a neighbor or not
                if dist < nm_cutoff and idx != jdx: #if atoms not the same and within 2A, neighbors
                    num_neighbors[idx] += 1
                elif elem2 > 30 and dist < m_cutoff: #if nonmetal atom has atomic number >30, allow 2.5A for neighbors
                    num_neighbors[idx] += 1
            else: #for metals, if within 2.5A, neighbors
                if dist < m_cutoff:
                    num_neighbors[idx] += 1
    num_neighbors = num_neighbors[num_neighbors != 0] #get rid of all the zero entries in the num_neighbors array
    if len(num_neighbors) < np.count_nonzero(elements == 6): #if getting rid of zeros in num_neighbors got rid of a C
        return 0.0 #then that carbon had no neighbors
    return np.min(num_neighbors) #want to know the carbon with the least neighbors

In [4]:
fewest_bonds = []

for idx, data in tqdm(enumerate(tmqm), total=len(tmqm)):
    fewest_bonds.append(fewestBonds(data))
    
fewest_bonds = np.array(fewest_bonds)

100%|██████████| 86665/86665 [08:16<00:00, 174.69it/s]


In [5]:
np.count_nonzero(fewest_bonds == 0)

0

In [6]:
np.count_nonzero(fewest_bonds == 1)

130

We see that there are 130 structures that have carbons with only one neighbor. We determine if these structures are erroneous or not by hand, by inspecting the CSD entry for their CSD code.

In [20]:
no_neighbor_idxs = np.where(fewest_bonds == 1)[0]

no_neighbor_csds = [tmqm[idx].info['CSD_code'] for idx in no_neighbor_idxs]
np.array(no_neighbor_csds)

array(['GUMYAP', 'HIWGEZ', 'MQUPTI', 'KASSAY', 'PESFAU', 'LACVIV',
       'JIBWUM', 'ACIPTI', 'MCPTIC', 'NONSIT', 'FAHXUI', 'CFMNTV',
       'WEKDAR', 'CPTLVB', 'CEXLAS', 'ZURKOM', 'NEBQEQ', 'DAMTIU',
       'ZACTAY', 'GEDCEX', 'ZICWIR', 'BAPMAG', 'JEBPUB', 'CPMONC',
       'KORGED', 'GOWDAX', 'MCTPCR', 'TEFZOT', 'ROJVOB', 'XEDREE',
       'JAWKUN', 'YAJPII', 'MAMYRE', 'VAGCAH', 'FOBVOH', 'GIDSOB',
       'VERKIM', 'GEMJOX', 'FICMAF', 'NEBQIU', 'XZODMN', 'GINGUF',
       'PABSUG', 'GAJYEY', 'GAJXOH', 'KUBPED', 'NEMGOE', 'JELPEV',
       'AHIDEB', 'FOCLUE', 'XILVUL', 'OGONON', 'FAVFEN', 'CAHYIT',
       'TLSCRU', 'ZAVLIR', 'JUFCAO', 'VAJVOR', 'SIVREV', 'FACCES',
       'IDOZOQ', 'RANKOG', 'ECUCEK', 'CAHMIH', 'ROBMAW', 'VIRQIW',
       'JOGVEG', 'CORLUQ', 'SURLEX', 'HEPHIT', 'ACMPNI', 'YUGTUP',
       'IPTCNI', 'HEXDUJ', 'PHMTPT', 'LATNID', 'PILWUC', 'BAVLIT',
       'MPPNNI', 'SUGWEW', 'SERGIF', 'NIBZUT', 'TOFMPD', 'NIXANT',
       'ACPXNI', 'CDBRNI', 'NABGON', 'AESTPT', 'KULJIL', 'TANS

All of the structures above have missing hydrogens on their CSD pages, except for:
- OGONON, ROJVOB, SIVREV, and IDOZOQ, which have C- ligands
- GAJXOH and GAJYEY, which have C ligands
- GUMYAP and HIWGEZ, which do not appear to have any erroneous features.

This method then screens out some invalid structures, but it is noted that this method requires manual checking, and cannot catch missing hydrogens in ring structures, or on any non-terminal carbons that have more than one bond. Thus, there may be more erroneous structures that this method did not remove.

In order to find some more of these structures, we check all of the structures that are completely missing hydrogens.

In [8]:
no_H = []

for idx, data in tqdm(enumerate(tmqm), total=len(tmqm)):
    if 1 not in data.get_atomic_numbers(): no_H.append(idx)
        
print(len(no_H))

100%|██████████| 86665/86665 [00:00<00:00, 168090.24it/s]

64


We again inspect the structures manually to see if there is anything erroneous.

In [9]:
noH_csds = [tmqm[idx].info['CSD_code'] for idx in no_H]
np.array(noH_csds)

array(['LERBUF', 'BIDPAG', 'LACVIV', 'CPTPHF', 'REJZEL', 'MPYHGB',
       'RIGGIY', 'MEGUHG', 'HISHGC', 'IETHSH', 'FUSYAT', 'PHGBAN',
       'SANLEY', 'FMTPHG', 'MECBHG', 'RAFJUD', 'CUQCUM', 'AQEPOB',
       'BINMIU', 'DITPHG', 'HGPARO', 'CMEEAM', 'BTETHG', 'HGCTOX',
       'BSHGCL', 'PHGTFM', 'BEGVEO', 'SURHGC', 'DUWMIR', 'PASHGB',
       'AQEPUH', 'BIJDAZ', 'AQEPIV', 'EFUGAO', 'BICMUV', 'FMHGAZ',
       'MERSET', 'METHGC', 'AQEQAO', 'GERCOW', 'HGTXZO', 'SQUIHG',
       'TESZUM', 'CPYHGQ', 'FMHGIC', 'HIXSOX', 'MPYHGN', 'BETWIG',
       'PHGCAN', 'RETWOC', 'POKREN', 'QEGQID', 'HGTHUR', 'IPTCHG',
       'PHGFAN', 'PENNHG', 'POKRAJ', 'COYRAK', 'HGACAM', 'JATVIJ',
       'CYHGTF', 'JAPROH', 'GUKHEZ', 'POKQUC'], dtype='<U6')

We see that, of the above, the following structures are fine:
- POKQUC, POKREN, POKRAJ, FMHGIC, BIDPAG, FMTPHG, CYHGTF, RIGGIY, METHGC, BETWIG, CUQCUM, AQEPIV, AQEPUH, AQEQAO, AQEPOB, BIJDAZ, GUKHEZ, HIXSOX, and GERCOW

We now remove all of the valid structures from the invalid list, and create a definitive list of the structures removed.

In [28]:
#list of all removed structures
CSDs_nonconverged = ['PIFFEP', 'NASBOA', 'LIKBIS']

noH_fine_structures = ['POKQUC', 'POKREN', 'POKRAJ', 'FMHGIC', 'BIDPAG',
                       'FMTPHG', 'CYHGTF', 'RIGGIY', 'METHGC', 'BETWIG',
                       'CUQCUM', 'AQEPIV', 'AQEPUH', 'AQEQAO', 'AQEPOB',
                       'BIJDAZ', 'GUKHEZ', 'HIXSOX', 'GERCOW']

no_H_CSDs = list(set(noH_csds) - set(noH_fine_structures))

no_neighbors_fine_structures = ['OGONON', 'ROJVOB', 'SIVREV', 'IDOZOQ',
                                'GAJXOH', 'GAJYEY', 'GUMYAP', 'HIWGEZ']

no_neighbors_CSDs = list(set(no_neighbor_csds) - set(no_neighbors_fine_structures))

removed_structures = set(no_H_CSDs + no_neighbors_CSDs)
np.array(list(removed_structures)), len(removed_structures)

(array(['DUGMIB', 'HGPARO', 'GEDCEX', 'HEXDUJ', 'JIBWUM', 'IETHSH',
        'ZICWIR', 'COTNUU', 'PILWUC', 'MCTPCR', 'NEBQIU', 'VIYKET',
        'FAHXUI', 'CDBRNI', 'FOCLUE', 'JEBPUB', 'MPYHGN', 'MCPTIC',
        'TLSCRU', 'PHMTPT', 'ZAVLIR', 'FACCES', 'NABGON', 'CUVYUN',
        'TINVIV', 'ACMPNI', 'LERBUF', 'BICMUV', 'IPTCNI', 'DIXCES',
        'ROBMAW', 'BAVLIT', 'CAHYIT', 'CMEEAM', 'SURHGC', 'DITPHG',
        'BSHGCL', 'NEMGOE', 'CAHMIH', 'PHGFAN', 'CPMONC', 'GOWDAX',
        'JELPEV', 'MEBCUU', 'PHGTFM', 'ECUCEK', 'AZAVUS', 'NIBZUT',
        'HMBPCU', 'LACVIV', 'DUWMIR', 'KETMIF', 'SQUIHG', 'TEFZOT',
        'SUCGOM', 'BAPMAG', 'FAVFEN', 'PENNHG', 'SURLEX', 'JAWKUN',
        'NONSIT', 'YUGTUP', 'HGTHUR', 'IPTCHG', 'ACPXNI', 'CORLUQ',
        'MQUPTI', 'FOBVOH', 'PABSUG', 'DAMTIU', 'TESZUM', 'GOFYIJ',
        'ZURKOM', 'VAGCAH', 'DAYYUX', 'DCMPPD', 'COWWAM', 'GINGUF',
        'HEPHIT', 'VAJVOR', 'PDTCNI', 'GEMJOX', 'PHGCAN', 'DAKZEU',
        'KULJIL', 'TOFMPD', 'DEFJUT', 'CETEPT', 

It is seen that we remove 155 structures, which are listed above, in addition to the three nonconverged structures. To access information about these structures, see their CSD entry, or use a command similar to the following to get the ASE Atoms representation. Merely change the `csd` variable to the CSD code of interest to view a different structure.

In [25]:
csd = 'ZACTAY'
tmqm[outlier_idxs[np.where(np.array(outlier_csds) == csd)[0][0]]]

Atoms(symbols='WBrON2C11H8', pbc=False)

These 158 structures were removed before generating the final version of tmQM_wB97MV. The final, complete datasets can be found in the `all_data` directory, as ASE dbs. These dbs contain all of the properties in tmQM for the tmQM datasets, as well as the targets used for ML training, and a numerical SID identifier that is unique to each structure for a given dataset. For tmQM_wB97MV, only the recomputed energy and the targets used for training are included in the dbs. An ASE db of these 158 removed structures is also included, which contains those structures' properties from tmQM along with their recomputed energies, if applicable. For an example of how to read and use an ASE db, see the following cell.

In [35]:
from ase.db import connect
from ase.io import read
db = connect('../all_data/removed_structures/removed_structures.db')

In [41]:
csd = 'PIFFEP'
for idx in trange(1, len(db)+1):
    row = db.get(id=idx)
    if row.data['CSD_code'] == csd:
        structure = read('../all_data/removed_structures/removed_structures.db@id='+str(idx))[0]
        structure.info = row.data
        
structure, structure.info

100%|██████████| 158/158 [00:00<00:00, 558.61it/s]


(Atoms(symbols='CoSi8O16C4H7', pbc=False),
 {'CSD_code': 'PIFFEP',
  'q': 0,
  'S': 0,
  'Stoichiometry': 'C4H7CoO16Si8',
  'MND': 5,
  'Electronic_E': -5058.53082,
  'Dispersion_E': -0.097131,
  'Dipole_M': 0.2499,
  'Metal_q': -1.37603,
  'HL_Gap': 0.18321,
  'HOMO_Energy': -0.25066,
  'LUMO_Energy': -0.06745,
  'Polarizability': 325.114085,
  'Recomputed_E': 'NOT CONVERGED'})

In general, one obtains the structures by using the `ase.io.read` command on the db, while also specifying an index (starting from 1). The data in the ASE db is retrieved by finding the row in the database, via using the `db.get` command with the same index, which can then be appended to the ASE Atoms object by setting it as the `info` dictionary, or used in other applications.